###  Docker Compose

<b>O que é o Docker Compose?</b>

- O Docker Compose é uma ferramenta para rodar múltiplos containers;

- Teremos apenas um arquivo de configuração, que orquestra totalmente esta situação;

- É uma forma de rodar múltiplos builds e runs com um comando;

- Em projetos maiores é essencial o uso do Compose;

<b>Instalando docker compose no Linux</b>

- Usuários do Linux ainda não possuem a ferramenta que utilizaremos nesta seção;

- Vamos seguir as instruções de: https://docs.docker.com/compose/install/

- O docker compose é essencial para atingirmos os nossos objetos no curso;

<b>Criando nosso primeiro Compose</b>

- Primeiramente vamos criar um arquivo chamado ```docker-compose.yml``` na raiz do projeto;

- Este arquivo vai coordenar os containers e imagens, e possui algumas chaves muito utilizadas;

- ```version```: versão do Compose;

- ```services```: Containers/serviços que vão rodar nessa aplicação;

- ```volumes```: Possível adição de volumes;

```yaml
version: '3.3'

services:
  db: 
    image: mysql:5.7
    ports:
      - 3306:3306 
    volumes:
      - db_data:/var/lib/mysql
    restart: always
    environment:
      MYSQL_ROOT_PASSWORD: wordpress
      MYSQL_DATABASE: wordpress
      MYSQL_USER: bruno
      MYSQL_PASSWORD: secret

  wordpress:
    depends_on: 
      - db
    image: wordpress:latest
    ports:
      - "80:80"
    restart: always
    environment: 
      WORDPRESS_DB_HOST: db:3306
      WORDPRESS_DB_USER: bruno
      WORDPRESS_DB_PASSWORD: secret
      WORDPRESS_DB_NAME: wordpress

volumes:
  db_data: {}
```

<b>Rodando o Compose</b>

- Para rodar nossa estrutura em Compose vamos utilizar o comando: ```docker compose up```;

- Isso fará com que as instruções no arquivo sejam executadas;

- Da mesma forma que realizamos os builds e também os runs;

- Podemos parar o Compose com ```ctrl+c``` no terminal;

In [ ]:
! docker compose up

<b>Compose em background</b>

- O Compose também pode ser executado em modo ```detached```;

- Para isso vamos utilizar a ```flag -d``` no comando;

- E então os containers estarão rodando em background;

- Podemos ver sua execução com ```docker ps```;

In [ ]:
! docker compose up -d

<b>Parando o Compose</b>

- Podemos parar o Compose que roda em background com:```docker compose down```;

- Desta maneira o serviço para e temos os containers adicionados no ```docker ps -a```;

In [ ]:
! docker compose down

<b>Variáveis de ambiente</b>

- Podemos definir variáveis de ambiente para o Docker Compose;

- Para isso vamos definir um arquivo base em ```env_file```;

- As variáveis podem ser chamadas pela sintaxe: ```${VARIAVEL}```

- Esta técnica é útil quando o dado a ser inserido é sensível/não pode ser compartilhado, como uma senha;

<b>Arquivo db.env</b>

```env
MYSQL_ROOT_PASSWORD=wordpress
MYSQL_DATABASE=wordpress
MYSQL_USER=bruno
MYSQL_PASSWORD=secret
```

<b>Arquivo wp.env</b>

```env
WORDPRESS_DB_HOST=db:3306
WORDPRESS_DB_USER=bruno
WORDPRESS_DB_PASSWORD=secret
WORDPRESS_DB_NAME=wordpress
```

<b>Arquivo docker-compose.yaml</b>

```yaml
version: '3.3'

services:
  db: 
    image: mysql:5.7
    ports:
      - 3306:3306 
    volumes:
      - db_data:/var/lib/mysql
    restart: always
    env_file:
      - ./config/db.env

  wordpress:
    depends_on: 
      - db
    image: wordpress:latest
    ports:
      - "80:80"
    restart: always
    env_file:
      - ./config/wp.env

volumes:
  db_data: {}
```

<b>Redes no Compose</b>

- O Compose cria uma rede básica Bridge entre os containers da aplicação;

- Porém podemos isolar as redes com a chave ```networks```;

- Desta maneira podemos conectar apenas os containers que optarmos;

- E podemos definir drivers diferentes também;

```yaml
version: '3.4'
services:
  db:
    image: mysql:5.7.22
    command: mysqld --default_authentication_plugin=mysql_native_password
    environment:
      TZ: America/Sao_Paulo
      MYSQL_ROOT_PASSWORD: docker
      MYSQL_USER: docker
      MYSQL_PASSWORD: docker
      MYSQL_DATABASE: wordpress
    ports:
      - 3308:3306
    networks:
      - wordpress-network
  wordpress:
    image: wordpress:latest
    volumes:
      - ./config/php.conf.uploads.ini:/usr/local/etc/php/conf.d/uploads.ini
      - ./wp-app:/var/www/html
    environment:
      TZ: America/Sao_Paulo
      WORDPRESS_DB_HOST: db
      WORDPRESS_DB_NAME: wordpress
      WORDPRESS_DB_USER: root
      WORDPRESS_DB_PASSWORD: docker
    ports:
      - 80:80
    networks:
      - wordpress-network
networks:
    wordpress-network:
      driver: bridge

```

<b>Vamos incluir o projeto no Compose</b>

- Agora vamos inserir o nosso projeto da última seção no Compose;

- Para verificar na prática como fazer uma transferência de Dockerfiles para Docker Compose!

```yaml
version: '3.3'

services:
  db:
    image: mysql
    restart: always
    env_file:
      - ./config/db.env
    ports:
      - "3306:3306"
    networks:
      - wordpress-network
    volumes:
      - ./mysql/schema.sql:/docker-entrypoint-initdb.d/init.sql
  
  flask:
    depends_on: 
      - db
    image: bruiglesias/flask-image
    ports:
      - "5000:5000"
    restart: always
    networks: 
      - wordpress-network

networks:
  wordpress-network:

```

In [ ]:
! docker compose up -d

<b>Build no Compose</b>

- Podemos gerar o build durante o Compose também;

- Isso vai eliminar o processo de gerar o build da imagem a cada atualização;

- Vamos ver na prática!

```yaml
version: '3.3'

services:
  db:
    build: ./mysql/
    restart: always
    env_file:
      - ./config/db.env
    ports:
      - "3306:3306"
    networks:
      - wordpress-network
    volumes:
      - ./mysql/schema.sql:/docker-entrypoint-initdb.d/init.sql
  
  flask:
    depends_on: 
      - db
    build: ./flask/
    ports:
      - "5000:5000"
    restart: always
    networks: 
      - wordpress-network

networks:
  wordpress-network:
```

In [ ]:
! docker compose up -d

<b>Bind mount no Compose</b>

- O volume de Bind Mount garante atualização em tempo real dos arquivos do container;

- Podemos configurar nosso projeto de Compose para utilizar esta funcionalidade também;

- Vamos ver na prática!

```yaml
version: '3.3'

services:
  db:
    build: ./mysql/
    restart: always
    env_file:
      - ./config/db.env
    ports:
      - "3306:3306"
    networks:
      - wordpress-network
    volumes:
      - ./mysql/schema.sql:/docker-entrypoint-initdb.d/init.sql
  
  flask:
    depends_on: 
      - db
    build: ./flask/
    ports:
      - "5000:5000"
    restart: always
    volumes:
      - /home/iglesias/Documentos/curso_docker/compose/5_bind/flask:/app
    networks: 
      - wordpress-network

networks:
  wordpress-network:
```

<b>Verificando o que tem no Compose</b>

- Podemos fazer a verificação do compose com: ```docker-compose ps```

- Receberemos um resumo dos serviços que sobem ao rodar o compose;

- Desta maneira podemos avaliar rapidamente o projeto;

In [ ]:
! docker-compose ps